# HW4 Trees and Forests

Official instructions:

https://www.cs.tufts.edu/cs/135/2025f/hw4.html

This is the *starter code* notebook for Problem 2 specifically, looking at sentiment classification using bag-of-words features of product reviews.

In [136]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [137]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble

In [138]:
# From the starter code
from pretty_print_sklearn_tree import pretty_print_sklearn_tree

In [139]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

# Load all data from train/valid/test

In [140]:
DATA_DIR = os.path.join("data_product_reviews")

### Load training

In [141]:
x_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv.zip'))
y_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
x_tr_NF = np.minimum(x_tr_df.values, 1.0).copy()
y_tr_N = y_tr_df.values[:,0].copy()

print("Training data")
print("x_tr_NF.shape: %s" % str(x_tr_NF.shape))
print("y_tr_N.shape : %s" % str(y_tr_N.shape))
print("mean(y_tr_N) : %.3f" % np.mean(y_tr_N))

Training data
x_tr_NF.shape: (6346, 7729)
y_tr_N.shape : (6346,)
mean(y_tr_N) : 0.500


### Load validation set

In [142]:
x_va_df = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv.zip'))
y_va_df = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv'))

x_va_TF = np.minimum(x_va_df.values, 1.0).copy()
y_va_T = y_va_df.values[:,0].copy()

print("Validation data")
print("x_va_TF.shape: %s" % str(x_va_TF.shape))
print("y_va_T.shape : %s" % str(y_va_T.shape))
print("mean(y_va_T) : %.3f" % np.mean(y_va_T))

Validation data
x_va_TF.shape: (792, 7729)
y_va_T.shape : (792,)
mean(y_va_T) : 0.490


### Load test set 

In [143]:
x_te_df = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv.zip'))
y_te_df = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv'))

x_te_TF = np.minimum(x_te_df.values, 1.0).copy()
y_te_T = y_te_df.values[:,0].copy()

print("Heldout Test data")
print("x_te_TF.shape: %s" % str(x_te_TF.shape))
print("y_te_T.shape : %s" % str(y_te_T.shape))
print("mean(y_te_T) : %.3f" % np.mean(y_te_T))

Heldout Test data
x_te_TF.shape: (793, 7729)
y_te_T.shape : (793,)
mean(y_te_T) : 0.515


### Load vocabulary as a list of strings

In [144]:
vocab_list = x_tr_df.columns.tolist()

print("total vocab size", len(vocab_list))

total vocab size 7729


In [145]:
for word in vocab_list[:8]:
    print(word)
print("...")
for word in vocab_list[-8:]:
    print(word)

good
great
time
book
don't
work
i_have
read
...
never_get
i'd_like
loves_it
an_author
nomin
could_give
bad_but
gap


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [146]:
xall_LF = np.vstack([x_tr_NF, x_va_TF])
yall_L = np.hstack([y_tr_N, y_va_T])

In [147]:
valid_indicators_L = np.hstack([
    -1 * np.ones(y_tr_N.size), # -1 means never include this example in any test split
    0  * np.ones(y_va_T.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])

In [148]:
# Create splitter object using Predefined Split
# Will be used later by all hyperparameter searches

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

**Tip: Perform fixed validation using GridsearchCV by passing a predefined split**

Although `GridsearchCV` is mostly used for cross validation (as the name implies), we can still have it perform fixed validation by passing a predefined split as the `cv` parameter when calling it. And a `PredefinedSplit` object is how we construct a predefined split. We can pass the desired splitting scheme to `PredefinedSplit` to control how the data is split into train/validation data. We will illustrate this using a simple dataset.

In [149]:
X_LF = np.asarray([[1, 3],
                  [4, 4],
                  [8, 2],
                  [4, 3],
                  [7, 7]])
Y_L = np.asarray([4, 8, 10, 7, 14])

Let's say that we want to use the examples indexed at 0, 1, and 2 as the training set and the examples indexed at 3 and 4 as the validation set. We can specify this using a `PredefinedSplit`:

In [150]:
splitter = sklearn.model_selection.PredefinedSplit(test_fold=[-1, -1, -1, 0, 0])

Here, `-1` means to never include the example at this position in any validation fold (i.e. only use them for training), and `0` means to use the example at this position in the number 0 validation fold. But since we are doing fixed validation, there is only one validation fold (number 0).

By passing the defined `splitter` as the `cv` parameter into sklearn's `GridsearchCV` object, we can ask it to do fixed validation using the split we defined, instead of the default cross validation.

In hyperparameter searching, `GridsearchCV` will train the models using the examples corresponding to `-1`s in `test_fold` and validate using the examples corresponding to `0`s.

In [151]:
for _, (train_index, test_index) in enumerate(splitter.split()):
    print(f"Training indices: {train_index}")
    print(f"Validation indices: {test_index}")

Training indices: [0 1 2]
Validation indices: [3 4]


For more details, see the `PredefinedSplit` doc [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.PredefinedSplit.html). 

Now, utilize the `my_splitter` that we defined above for our sentiment classification task to perform fixed validation using `GridsearchCV` in the following problems.

# Decision Trees

## Step 1A: Train a simple tree with depth 3

In [152]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='entropy', 
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1, 
    random_state=101)

### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [153]:
simple_tree.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=101)

### **Figure 1: Print Tree** 

Use a helper function from the starter code

In [154]:
pretty_print_sklearn_tree(simple_tree, feature_names=vocab_list)

The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['waste'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.422 (4199 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.033 (210 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (277 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (14 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1413 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (142 total training examples)
    N Decision: X['in_all'] <= 0.50?
      Y 

## 1B : Find best Decision Tree with grid search

In [155]:
# Construct the default predictor
# Any hyperparameters here may be overridden by the hyperparameter grid

base_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='entropy',
    max_depth=8,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=101,
    )

In [156]:
tree_hyperparameter_grid_by_name = dict(
    max_depth=[2, 8, 32, 128],
    min_samples_leaf=[1, 3, 9],
    random_state = [101],
    )

**TODO Build the Grid Search** in the next coding cell

Follow the 1B instructions carefully:

* Set `scoring='balanced_accuracy'`, since our target metric is balanced accuracy
* Set `cv=my_splitter` (as in starter code), so you can use the predefined split we defined earlier.
* Set `return_train_score=True`, since we want training set scores as well as valdiation set scores (which will be called 'test' by the searcher)
* Set `refit=False`, because we only want fits on `x_tr_NF`

In [157]:
tree_grid_searcher = sklearn.model_selection.GridSearchCV(
    estimator=base_tree,
    param_grid=tree_hyperparameter_grid_by_name,
    scoring='balanced_accuracy',
    cv=my_splitter,
    return_train_score=True,
    refit=False
)

### Do the search!


In [158]:
start_time_sec = time.time()
tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

### Build dataframe of results

Move the results of grid search into a nice pandas data frame.

In [159]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    # Will execute if you called fit on tree_grid_searcher succesfully
    tree_search_results_df = pd.DataFrame(tree_grid_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        tree_search_results_df.shape[0], elapsed_time_sec))

Grid search of  12 configurations done after  103.9 sec


### Display search results

This block will make a pretty printed table of the results of your grid search

In [160]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    tree_keys = ['param_max_depth', 'param_min_samples_leaf']
    tree_search_results_df.sort_values(tree_keys, inplace=True)
    tree_search_results_df[tree_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']]

In [161]:
print("Printing a dict of the best hyperparameters")
try:
    print(tree_grid_searcher.best_params_)
except AttributeError:
    print("AttributeError happens if you did not complete earlier 'Do the search' todo")

Printing a dict of the best hyperparameters
{'max_depth': 128, 'min_samples_leaf': 9, 'random_state': 101}


### Build the best decision tree

**TODO Build the Best Tree on the training set** in the next coding cell

This is necessary so you have the specific best performing tree in your workspace.

Although you fit many trees in the search, they were not stored, so we need to recreate the best one.

Hint: Just feed the best hyperparameters as keyword args to construct the tree and fit it to the training data.

In [162]:
best_tree = base_tree.set_params(**tree_grid_searcher.best_params_)
best_tree.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(criterion='entropy', max_depth=128, min_samples_leaf=9,
                       random_state=101)

### Interpret the best decision tree

In [163]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)

The binary tree structure has 775 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    2 are leaves
- depth   4 has   12 nodes, of which    3 are leaves
- depth   5 has   18 nodes, of which    6 are leaves
- depth   6 has   24 nodes, of which   14 are leaves
- depth   7 has   20 nodes, of which    8 are leaves
- depth   8 has   24 nodes, of which    9 are leaves
- depth   9 has   30 nodes, of which   18 are leaves
- depth  10 has   24 nodes, of which    8 are leaves
- depth  11 has   32 nodes, of which   17 are leaves
- depth  12 has   30 nodes, of which   16 are leaves
- depth  13 has   28 nodes, of which   15 are leaves
- depth  14 has   26 nodes, of which   13 are leaves
- depth  15 has   26 nodes, of which   15 are leaves
- depth  16 has   22 nodes, of which   10 are leaves
- depth  17 has   24 nodes, of which   11 are leaves
- dep

# Random forests!

## 1C: Train a random forest with default settings

In [164]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)

### Fit the forest

**TODO Train on the training set** in the next coding cell

In [165]:
simple_forest.fit(x_tr_NF, y_tr_N)

RandomForestClassifier(criterion='entropy', max_depth=3, random_state=101)

## 1D: Best Random Forest via grid search

Follow the instructions and using what you learn above to finish this step.

This block might take 2-10 minutes. (Takes about 2 min on staff Macbook laptops from before 2020.)

If yours runs significantly longer, try this out on Google Colab instead.

In [166]:
base_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_depth=16,
    min_samples_split=2,
    min_samples_leaf=1)


In [167]:
forest_hyperparameter_grid_by_name = dict(
    max_features=[3, 10, 33, 100, 333],
    max_depth=[16, 32],
    min_samples_leaf=[1],
    n_estimators=[100],
    random_state=[101],
    )

In [174]:
# Make sure you point it to our predefined split
forest_searcher = sklearn.model_selection.GridSearchCV(
    estimator=base_forest,
    param_grid=forest_hyperparameter_grid_by_name,
    scoring='balanced_accuracy',
    cv=my_splitter,
    return_train_score=True,
    refit=False,
)

### Do the search!

In [175]:
start_time_sec = time.time()
forest_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

### Build dataframe of results

In [176]:
# Convert grid search results to a DataFrame
forest_results_df = pd.DataFrame(forest_searcher.cv_results_)
# Sort by best validation score
forest_results_df = forest_results_df.sort_values(
    by='mean_test_score', ascending=False
)
# Show top few rows
forest_results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_n_estimators,param_random_state,params,split0_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,mean_train_score,std_train_score
6,2.4921,0.0,0.1480,0.0,32,10,1,100,101,"{'max_depth': 32, 'max_features': 10, 'min_sam...",0.8620,0.8620,0.0,1,0.9724,0.9724,0.0
8,15.8675,0.0,0.0691,0.0,32,100,1,100,101,"{'max_depth': 32, 'max_features': 100, 'min_sa...",0.8469,0.8469,0.0,2,0.9694,0.9694,0.0
1,1.6869,0.0,0.0786,0.0,16,10,1,100,101,"{'max_depth': 16, 'max_features': 10, 'min_sam...",0.8458,0.8458,0.0,3,0.9272,0.9272,0.0
7,6.6287,0.0,0.0820,0.0,32,33,1,100,101,"{'max_depth': 32, 'max_features': 33, 'min_sam...",0.8422,0.8422,0.0,4,0.9635,0.9635,0.0
3,9.1792,0.0,0.0568,0.0,16,100,1,100,101,"{'max_depth': 16, 'max_features': 100, 'min_sa...",0.8409,0.8409,0.0,5,0.9130,0.9130,0.0


### Display search results

In [178]:
print("Best parameters:", forest_searcher.best_params_)
print("Best balanced accuracy: %.3f" % forest_searcher.best_score_)

Best parameters: {'max_depth': 32, 'max_features': 10, 'min_samples_leaf': 1, 'n_estimators': 100, 'random_state': 101}
Best balanced accuracy: 0.862


### Build the best random forest using the best hyperparameters found

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [179]:
# Build & train best forest (train set only)
best_forest = base_forest.set_params(**forest_searcher.best_params_)
best_forest.fit(x_tr_NF, y_tr_N)

RandomForestClassifier(criterion='entropy', max_depth=32, max_features=10,
                       random_state=101)

## Table 2: Comparison of methods on the bag-of-words to sentiment classification task.

Remember, in your workspace, you should have the following models defined

* `simple_tree`, from 1A
* `best_tree`, from 1B
* `simple_forest`, from 1C
* `best_forest`, from 1D


Please report **balanced accuracy** on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|simple Tree	| 1 | 1 | 0.123	|0.456	|0.890|
|best Tree	|1 | 1 | 0.123	|0.456	|0.890|
|simple RandomForest	|1 | 1 | 0.123	|0.456	|0.890|
|best RandomForest	|1 | 1 | 0.123	|0.456	|0.890|

In [ ]:
# Compute balanced accuracy for each dataset and model

results = []

# 1. Simple Decision Tree
yhat_tr = simple_tree.predict(x_tr_NF)
yhat_valid = simple_tree.predict(x_va_TF)
yhat_test = simple_tree.predict(x_te_TF)
results.append({
    'Model': 'Simple Decision Tree',
    '# Trees': 1,
    'max_depth': simple_tree.max_depth,
    'Train BA': sklearn.metrics.balanced_accuracy_score(y_tr_N, yhat_tr),
    'Valid BA': sklearn.metrics.balanced_accuracy_score(y_va_T, yhat_valid),
    'Test BA': sklearn.metrics.balanced_accuracy_score(y_te_T, yhat_test)
})

# 2. Best Decision Tree
yhat_tr = best_tree.predict(x_tr_NF)
yhat_valid = best_tree.predict(x_va_TF)
yhat_test = best_tree.predict(x_te_TF)
results.append({
    'Model': 'Best Decision Tree',
    '# Trees': 1,
    'max_depth': best_tree.max_depth,
    'Train BA': sklearn.metrics.balanced_accuracy_score(y_tr_N, yhat_tr),
    'Valid BA': sklearn.metrics.balanced_accuracy_score(y_va_T, yhat_valid),
    'Test BA': sklearn.metrics.balanced_accuracy_score(y_te_T, yhat_test)
})

# 3. Simple Random Forest
yhat_tr = simple_forest.predict(x_tr_NF)
yhat_valid = simple_forest.predict(x_va_TF)
yhat_test = simple_forest.predict(x_te_TF)
results.append({
    'Model': 'Simple Random Forest',
    '# Trees': simple_forest.n_estimators,
    'max_depth': simple_forest.max_depth,
    'Train BA': sklearn.metrics.balanced_accuracy_score(y_tr_N, yhat_tr),
    'Valid BA': sklearn.metrics.balanced_accuracy_score(y_va_T, yhat_valid),
    'Test BA': sklearn.metrics.balanced_accuracy_score(y_te_T, yhat_test)
})

# 4. Best Random Forest
yhat_tr = best_forest.predict(x_tr_NF)
yhat_valid = best_forest.predict(x_va_TF)
yhat_test = best_forest.predict(x_te_TF)
results.append({
    'Model': 'Best Random Forest',
    '# Trees': best_forest.n_estimators,
    'max_depth': best_forest.max_depth,
    'Train BA': sklearn.metrics.balanced_accuracy_score(y_tr_N, yhat_tr),
    'Valid BA': sklearn.metrics.balanced_accuracy_score(y_va_T, yhat_valid),
    'Test BA': sklearn.metrics.balanced_accuracy_score(y_te_T, yhat_test)
})

# Make DataFrame
table2_df = pd.DataFrame(results)

# Round for clarity
table2_df = table2_df.round(3)

# Print nicely
print("Table 2. Balanced Accuracy of all Models (Train, Validation, Test)")
display(table2_df)

Table 2. Balanced Accuracy of all Models (Train, Validation, Test)


,Model,# Trees,max_depth,Train BA,Valid BA,Test BA
0,Simple Decision Tree,1,3,0.646,0.643,0.647
1,Best Decision Tree,1,128,0.844,0.724,0.735
2,Simple Random Forest,100,3,0.815,0.795,0.774
3,Best Random Forest,100,32,0.972,0.862,0.806
